# Notebook to store municipalities records (JSON) into bronze.municipalities table

Define constant variables (location, target entity)

In [0]:
%run ../../config/utils

In [0]:
TARGET_TABLE="brz_municipalities"

### Getting the municipalities (parquet)

In [0]:
df_municipalities=spark.read.parquet(municipalities_volume_path)
df_municipalities.createOrReplaceTempView("municipalities")

### Write into bronze.municipalities

In [0]:
spark.sql(f"""
  MERGE INTO {catalog_name}.{bronze_schema_name}.{TARGET_TABLE} AS target
  USING municipalities AS source
  ON target.DPTOMPIO = source.DPTOMPIO
  WHEN MATCHED AND (
      target.DPTO_CCDGO IS DISTINCT FROM source.DPTO_CCDGO OR
      target.MPIO_CCDGO IS DISTINCT FROM source.MPIO_CCDGO OR
      target.MPIO_CNMBR IS DISTINCT FROM source.MPIO_CNMBR OR
      target.MPIO_CCNCT IS DISTINCT FROM source.MPIO_CCNCT OR
      target.geometry   IS DISTINCT FROM source.geometry
  ) THEN
    UPDATE SET
      target.DPTO_CCDGO = source.DPTO_CCDGO,
      target.MPIO_CCDGO = source.MPIO_CCDGO,
      target.MPIO_CNMBR = source.MPIO_CNMBR,
      target.MPIO_CCNCT = source.MPIO_CCNCT,
      target.geometry   = source.geometry
  WHEN NOT MATCHED THEN
    INSERT (
      DPTOMPIO,
      DPTO_CCDGO,
      MPIO_CCDGO,
      MPIO_CNMBR,
      MPIO_CCNCT,
      geometry
    )
    VALUES (
      source.DPTOMPIO,
      source.DPTO_CCDGO,
      source.MPIO_CCDGO,
      source.MPIO_CNMBR,
      source.MPIO_CCNCT,
      source.geometry
    )
""")